# **Trabalho Prático 1 - Geometria Computacional**


### **Universidade Federal de Minas Gerais - Instituto de Ciências Exatas**

#### Departamento de Ciência da Computação

DCC207 - Algoritmos 2

Prof. Renato Vimieiro


  
**Alunos:** Daniel Barbosa, Frederico Baker e Victor Prates

O presente trabalho visa a aplicação de algoritmos de geometria computacional para solucionar um problema de classificação em aprendizado de máquina supervisionado. Em especial, o problema de classificação a ser abordado se refere à triagem de pacientes em um serviço de saúde. A meta é criar um modelo de classificação usando técnicas como envoltória convexa e detecção de interseções entre segmentos.

# 1. Envoltória convexa

# 2. Varredura linear

# 3. Verificação de separabilidade

# 4. Construção do modelo e classificador

# 5. Método para computação das métricas

# 6. Conclusão dos experimentos

# 7. Conclusão do relatório

## Implementando Primitivas

Primeiro, vamos implementar um classe Point para modelarmos os nossos data points no espaço 2D:

In [ ]:
class Point:

  def __init__(self, x, y):
    self.x = x
    self.y = y

Agora implementaremos uma classe Segment para modelarmos os segmentos de reta entre nossos data points no espaço 2D:

In [ ]:
class Segment:

   def __init__(self, orig, dest):
    self.orig = orig
    self.dest = dest

Agora vamos implementar uma função que calcula o produto vetorial de dois vetores com mesma origem, ou seja, que calcula a orientação de um primeiro vetor em relação a um segundo vetor (horária ou anti-horária):

In [ ]:
"""
calculates the orientation
(clockwise/counter-clockwise)
of a vector A in relation to a vector B
"""
def vectProd(vectA, vectB):
  return vectA.x * vectB.y - vectA.y * vectB.x

"""
calculates the orientation
(clockwise/counter-clockwise)
of a segment A in relation to a
segment B that has the same
origin as segment A, but ends at a point B
"""
def orientation(segmentA, pointB):

  vectA = Point(segmentA.dest.x - segmentA.orig.x, segmentA.dest.y - segmentA.orig.y)
  vectB = Point(pointB.x - segmentA.orig.x, pointB.y - segmentA.orig.y)

  return vectProd(vectA, vectB)


Aqui, vamos escrever uma função que testará se nosso código que calcula a orientação (horária ou anti-horária) de dois vetores com mesma origem produz as saídas corretas dado determinadas entradas

In [ ]:
def test_orientation():

    # Test 1: If p0p1 is oriented clockwise in relation to p0p2
    p0 = Point(0, 0)
    p1 = Point(1, 1)
    p2 = Point(1, 2)
    seg1 = Segment(p0, p1)
    assert orientation(seg1, p2) > 0, "Test 1 Failed"

    # Test 2: If p0p1 is oriented counter-clockwise in relation to p0p3
    p3 = Point(2, 0)
    assert orientation(seg1, p3) < 0, "Test 2 Failed"

    # Test 3: If p0p1 is colinear to p0p4
    p4 = Point(2, 2)
    assert orientation(seg1, p4) == 0, "Test 3 Failed"

    # Test 4: If p5p6 is colinear to p5p7 (all points y axis)
    p5 = Point(0, 0)
    p6 = Point(0, 1)
    seg2 = Segment(p5, p6)
    p7 = Point(0, 2)
    assert orientation(seg2, p7) == 0, "Test 4 Failed"

    # Test 5: If p8p9 is colinear to p8p10 (all points x axis)
    p8 = Point(0, 0)
    p9 = Point(1, 0)
    seg3 = Segment(p8, p9)
    p10 = Point(2, 0)
    assert orientation(seg3, p10) == 0, "Test 5 Failed"

    # Test 6: If p11p12 is oriented counter-clockwise in relation to p11p13 (all negative coordinates)
    p11 = Point(-2, -2)
    p12 = Point(-3, -3)
    seg4 = Segment(p11, p12)
    p13 = Point(-4, -2)
    assert orientation(seg4, p13) < 0, "Test 6 Failed"

    print("All Tests Passed!")

test_orientation()


All Tests Passed!


Agora, vamos implementar uma função que verifica se, dado um ponto e um segmento, o ponto está contido dentro dos intervalos de x e de y do segmento

In [ ]:
def pointOnSegment(segment, point):

  pointInXInterval = point.x >= min(segment.orig.x, segment.dest.x) and point.x <= max(segment.orig.x, segment.dest.x)
  pointInYInterval = point.y >= min(segment.orig.y, segment.dest.y) and point.y <= max(segment.orig.y, segment.dest.y)

  if pointInXInterval and pointInYInterval:
    return True

  return False


Aqui, vamos escrever uma função que testará se nosso código que verifica se um ponto está contido dentro dos intervalos de x e de y de um segmento produz as saídas corretas dado determinadas entradas

In [ ]:
def testPointOnSegment():

    # Test 1: Point on the segment
    p0 = Point(1, 1)
    p1 = Point(2, 2)
    seg1 = Segment(p0, p1)
    assert pointOnSegment(seg1, p1) == True, "Test 1 Failed"

    # Test 2: Point off the segment but on the line extended from the segment
    p2 = Point(3, 3)
    assert pointOnSegment(seg1, p2) == False, "Test 2 Failed"

    # Test 3: Point outside the line formed by the segment
    p3 = Point(2, 3)
    assert pointOnSegment(seg1, p3) == False, "Test 3 Failed"

    # Test 4: Point exactly at the start of the segment
    assert pointOnSegment(seg1, p0) == True, "Test 4 Failed"

    # Test 5: Point inside the segment, but not at start or end
    p4 = Point(1.5, 1.5)
    assert pointOnSegment(seg1, p4) == True, "Test 5 Failed"

    # Test 6: Point lying on the vertical segment
    p5 = Point(2, 0)
    p6 = Point(2, 3)
    p7 = Point(2, 1.5)
    seg2 = Segment(p5, p6)
    assert pointOnSegment(seg2, p7) == True, "Test 6 Failed"

    print("All Tests Passed!")

testPointOnSegment()


All Tests Passed!


#Algoritmo de Interseção de Segmentos

Aqui implementaremos um algoritmo que verifica se dois segmentos se interceptam:

In [ ]:
def segmentIntersect(segmentA, segmentB):

  d1 = orientation(segmentB, segmentA.orig)
  d2 = orientation(segmentB, segmentA.dest)
  d3 = orientation(segmentA, segmentB.orig)
  d4 = orientation(segmentA, segmentB.dest)

  print(d1, d2, d3, d4)

  if ((d1 > 0 and d2 < 0) or (d1 < 0 and d2 > 0)) and ((d3 > 0 and d4 < 0) or (d3 < 0 and d4 > 0)):
    return True
  elif d1 == 0 and pointOnSegment(segmentB, segmentA.orig):
    return True
  elif d2 == 0 and pointOnSegment(segmentB, segmentA.dest):
    return True
  elif d3 == 0 and pointOnSegment(segmentA, segmentB.orig):
    return True
  elif d4 == 0 and pointOnSegment(segmentA, segmentB.dest):
    return True
  else:
    return False


In [ ]:
def testSegmentIntersect():
    # Test 1: Segments that intersect
    p0 = Point(1, 1)
    p1 = Point(2, 2)
    p2 = Point(3, 2.5)
    seg1 = Segment(p0, p1)
    seg2 = Segment(p1, p2)
    assert segmentIntersect(seg1, seg2) == True, "Test 1 Failed"

    # Test 2: Segments that intersect
    p1 = Point(1, 1.5)
    p2 = Point(3, 1.5)
    p3 = Point(2, 2)
    p4 = Point(2, 1)
    seg1 = Segment(p1, p2)
    seg2 = Segment(p2, p3)
    assert segmentIntersect(seg1, seg2) == True, "Test 2 Failed"

    # Test 3: Segments that don't intersect (colinear)
    p1 = Point(1, 1.5)
    p2 = Point(3, 1.5)
    p3 = Point(4, 1.5)
    p4 = Point(5, 1.5)
    seg1 = Segment(p1, p2)
    seg2 = Segment(p3, p4)
    assert segmentIntersect(seg1, seg2) == False, "Test 3 Failed"

    # Test 4: Segments that intersect (colinear)
    p1 = Point(1, 1.5)
    p2 = Point(3, 1.5)
    p3 = Point(2, 1.5)
    p4 = Point(5, 1.5)
    seg1 = Segment(p1, p2)
    seg2 = Segment(p3, p4)
    assert segmentIntersect(seg1, seg2) == True, "Test 4 Failed"

    # Test 5: Segments that intersect on one point, perpendicular to each other (forming a right angle)
    p1 = Point(1, 1.5)
    p2 = Point(3, 1.5)
    p3 = Point(2, 2)
    p4 = Point(2, 1.5)
    seg1 = Segment(p1, p2)
    seg2 = Segment(p3, p4)
    assert segmentIntersect(seg1, seg2) == True, "Test 5 Failed"

    # Test 6: Segments that don't intersect, parallel to each other
    p1 = Point(1, 1)
    p2 = Point(3, 1)
    p3 = Point(1, 2)
    p4 = Point(3, 2)
    seg1 = Segment(p1, p2)
    seg2 = Segment(p3, p4)
    assert segmentIntersect(seg1, seg2) == False, "Test 6 Failed"

    # Test 7: Segments that intersect at one end
    p1 = Point(1, 1)
    p2 = Point(3, 1)
    p3 = Point(3, 1)
    p4 = Point(3, 2)
    seg1 = Segment(p1, p2)
    seg2 = Segment(p3, p4)
    assert segmentIntersect(seg1, seg2) == True, "Test 7 Failed"

    print("All Tests Passed!")

testSegmentIntersect()


-0.5 0.0 0 -0.5
1.0 -0.0 0.0 1.0
0.0 0.0 0.0 0.0
0.0 0.0 0.0 0.0
-0.5 0.5 1.0 0.0
-2 -2 2 2
2 0 0 2
All Tests Passed!


#Algoritmo de Envoltória Convexa

#Algoritmo de Varredura Linear

Nessa parte do trabalho implementaremos um algoritmo de varredura linear.